#
  - 4. Syndigo Mapping MachineLearning

In [1]:
def prepare_data_1():
    pgm = inspect.currentframe().f_code.co_name  
    start_time = time.time() 
    # Reading PIMMART data
    pim_gtin_mapped = pd.read_csv(DBFR + "PIM_Data_New_50_82Mn.csv", dtype=object) # (5082212, 24)

    SUBCOMS_excluded = ['INVENTORY VALUES','MISC SALES TRANS','MISC SALES TRANS (NON TAX)','CENTRAL SUPPLIES', \
                        'MISC SALES TRANS (TX TAXABLE)','COUPON','MISCELLANEOUS INCOME','CRV/EXCISE TAX NT/NF', \
                        'CRV DEPOSIT NT/F','CRV DEPOSIT T/NF','MISCELLANEOUS REFUNDS','CUSTOMER EXPENSES','ROUND UP COUPONS']
    pim_gtin_mapped = pim_gtin_mapped[~pim_gtin_mapped.SUBCOM_DSC.isin(SUBCOMS_excluded)]
    print(f'Before drop pim_gtin_mapped.shape {pim_gtin_mapped.shape}')
    print(f'After drop pim_gtin_mapped.shape {pim_gtin_mapped.shape}')
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD', 'ITM_ID', 'GTIN']:
        pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)
    
    # Reading Syndigo 259K data
    synd_ALL = pd.read_csv(DBFR + 'Syndigo_Final_ALL.csv', dtype='unicode') # 259k Syndigo Data
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD', 'GTIN', 'ITM_ID', 'PMY_DPT_CD']:
        synd_ALL[i] = synd_ALL[i].astype(np.float64)
    
    # Stripping spaces from all columns
    df_obj = synd_ALL.select_dtypes(['object'])
    synd_ALL[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    
    syndigo_mapped = synd_ALL
    pimmart = pim_gtin_mapped
    syndigo_mapped.drop_duplicates('GTIN', inplace = True)

    
    syndigo_mapped['ITEM_SUBCOM_text'] = \
    (syndigo_mapped.VND_ECOM_DSC + ' ' + syndigo_mapped.SUBCOM_DSC).fillna('').str.lower()
    #syndigo_mapped['Level 1'].value_counts()
    #print(f'syndigo_mapped.shape {syndigo_mapped.shape}') 
    #print(f'syndigo_mapped.info() {syndigo_mapped.info()}') 
    #print(f'syndigo_mapped.head() {syndigo_mapped.head()}') 
    #print(f'syndigo_mapped.columns {syndigo_mapped.columns}') 
    
    syndigo_mapped['l1_l2'] = syndigo_mapped['Level 1'] + ' + ' + syndigo_mapped['Level 2']
    #print(f"syndigo_mapped['l1_l2'].value_counts() {syndigo_mapped['l1_l2'].value_counts(dropna = False)}")    
    return syndigo_mapped, pim_gtin_mapped

In [50]:
def prepare_data_2(samp_frac = 1, embed_typ = 'tfidf'):
  
    start_time = time.time()
    pgm = inspect.currentframe().f_code.co_name 
    global syndigo_mapped, model
    print(f' samp_frac = {samp_frac}') 
    print (f'Before sample syndigo_mapped.shape {syndigo_mapped.shape }')
    syndigo_mapped = syndigo_mapped_bkup.copy()
    
    whole_frac = 1
    if samp_frac  < whole_frac :   syndigo_mapped = syndigo_mapped.sample(frac = samp_frac,  random_state=42)   
    print (f'After sample syndigo_mapped.shape {syndigo_mapped.shape }')    
    
    
    series = syndigo_mapped['l1_l2'].value_counts(ascending=True)
    if series.tolist():
       syndigo_mapped = syndigo_mapped.drop(syndigo_mapped[syndigo_mapped['l1_l2'].isin(series[series==1].index.tolist())].index)
       print (f'after drop syndigo_mapped.shape {syndigo_mapped.shape }')                     
    if embed_typ == 'tfidf': 
        print(f'tfidf embed_typ {embed_typ}') 
        vect = TfidfVectorizer(ngram_range = (1,2), max_features = 50000) 
        X = vect.fit_transform(syndigo_mapped.ITEM_SUBCOM_text) #scipy.sparse._csr.csr_matrix
    else:
        model = KeyedVectors.load_word2vec_format(DBFO + 'w2vmodel_053123_PIM_ALL.bin', binary=True)
        X = np.array(list(syndigo_mapped.ITEM_SUBCOM_text.apply(lambda x: get_item_vector(x.split(' ')))))
        print(f'w2vec embed_typ {embed_typ}') 
        
    l1_l2_id_map = dict(zip(syndigo_mapped['l1_l2'].fillna('Other').unique(), range(syndigo_mapped['l1_l2'].fillna('Other').nunique())))
    id_l1_l2_map = dict(zip(range(syndigo_mapped['l1_l2'].fillna('Other').nunique()), syndigo_mapped['l1_l2'].fillna('Other').unique()))
    y  = syndigo_mapped['l1_l2'].fillna('Other').map(l1_l2_id_map)
    y_list= list(y)
    y = pd.Series(list(filter (lambda z: y_list.count(z) > 1, y_list)))
    
    end_time = time.time()   
    desc = f' Elapse_time for "{pgm}".' 
    elapse_time (  start_time, end_time, desc)
    return  X, y, l1_l2_id_map, id_l1_l2_map

In [12]:
#Build item vectors
def get_item_vector(item_vocab):
    vect = np.zeros_like(model.get_vector('chips'))
    for word in item_vocab:
        if word in model:
            vect += model.get_vector(word)
    return vect#/max(1,len(item_vocab))

In [36]:
def prep_data_3(X, y):
   
   xs = pd.Series(list(X))
   data = {
        'ITEM_SUBCOM_text': xs,
        'L1L2_val' : y
    
    }                  
    
   df_merg= pd.DataFrame(data)    
    #df_merg = pd.merge(dfx, dfy, left_index=True, right_index=True)
   return df_merg

In [37]:
df_merg = prep_data_3(X, y)
df_merg.head()

,ITEM_SUBCOM_text,L1L2_val
0,"[-10.710463, 15.938267, -31.25719, 4.2918553, ...",0
1,"[-11.04187, 5.650813, -9.047808, 4.3786564, 8....",1
2,"[7.025075, 1.1739403, -12.235872, 1.8468916, -...",2
3,"[3.9563594, -1.2443018, -2.8453398, 4.597701, ...",3
4,"[2.3601267, 0.85085166, 0.1611628, 11.792536, ...",4


In [11]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier,  RidgeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec, KeyedVectors
from sklearn.model_selection import train_test_split
from datetime import date


from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score, f1_score
import re, time, inspect, pickle
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

pd.options.display.float_format = '{:20,.2f}'.format 
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# load  function of 'elapse_time'
path_code = 'C:\\users\\iny2819\\kroger\\Code\\'  
f_com_code = path_code + "com_code.py"
exec(compile(open(f_com_code , "rb").read(), f_com_code, 'exec' ))


In [5]:
import pandas as pd, numpy as np
DBFS = "/dbfs/FileStore/tables/OFFSHORE/"
DBFO = "/dbfs/FileStore/tables/OFFSHORE/"
DBFM = "/dbfs/FileStore/tables/MALLIK/"
DBFR = "/dbfs/FileStore/tables/OFFSHORE_RESULTS/"
path = 'C:\\users\\iny2819\\kroger\\Data\\'   
DBFS = path
DBFO = path
DBFM = path
DBFR = path


In [8]:
syndigo_mapped = prepare_data_1()

Before drop pim_gtin_mapped.shape (5061471, 24)
After drop pim_gtin_mapped.shape (5061471, 24)


In [9]:
syndigo_mapped, pim_gtin_mapped = prepare_data_1()
syndigo_mapped_bkup = syndigo_mapped.copy()

Before drop pim_gtin_mapped.shape (5061471, 24)
After drop pim_gtin_mapped.shape (5061471, 24)


In [52]:
embed_typ = ['W2vec']
embed_typ = ['tfidf']
#X, y, l1_l2_id_map, id_l1_l2_map = prepare_data_2(samp_frac = 0.3, embed_typ = embed_typ )
X, y, l1_l2_id_map, id_l1_l2_map = prepare_data_2(samp_frac = 0.3 )

 samp_frac = 0.3
Before sample syndigo_mapped.shape (77675, 25)
After sample syndigo_mapped.shape (77726, 25)
after drop syndigo_mapped.shape (77675, 25)
tfidf embed_typ tfidf
  Elapse_time for "prepare_data_2". It took 65.727653 seconds - 0hh:1mm:5ss.
 start time: Sep 01 2023 14:29:22  end time:  Sep 01 2023 14:30:28


In [47]:
def prep_data_3(X, y):
    dfx = pd.DataFrame(X.toarray())
    data = {'L1L2_val' : y }                  
    
    
    dfy = pd.DataFrame(data)    
    df_merg = pd.merge(dfx, dfy, left_index=True, right_index=True)
    return df_merg

In [53]:
type(X)

scipy.sparse._csr.csr_matrix

In [54]:
df_merg = prep_data_3(X, y)
df_merg.head()

MemoryError: Unable to allocate 28.9 GiB for an array with shape (77675, 50000) and data type float64

In [49]:
type(X)

numpy.ndarray

In [44]:
# conda install -c conda-forge pycaret

# setup the dataset
#grid = setup(data=X_subset, target= y_subset, html=False, silent=True, verbose=False)
grid = setup(data=df_merg, target= df_merg.columns[-1], train_size = 0.7, html=False, verbose=False)
# evaluate models and compare models
best = compare_models()
# report the best model
print(best)

TypeError: unhashable type: 'numpy.ndarray'

In [ ]:
# tune model hyperparameters
best_yune = tune_model( RidgeClassifier(), n_iter=200, choose_better=True)
# report the best model
print(best_tune)

In [ ]:
print(best_yune)

In [ ]:
df_y.value_counts( dropna = False)

In [ ]:
# tune model hyperparameters
best_tune = tune_model( LogisticRegressionCV(), n_iter=200, choose_better=True)
# report the best model
print(best_tune)

In [ ]:
y_subset.index

In [ ]:
df1=dfx.reset_index().drop('index', axis = 1) 
df1.head()

In [ ]:
df2=dfy.reset_index().drop('index', axis = 1)

df2.head()

In [ ]:
df3 = pd.merge(df1, df2, left_index=True, right_index=True)
df3.head()

In [ ]:
df3['Level 2'].value_counts(dropna = False)

In [ ]:
dfz['Level 2'].value_counts(dropna = True)

In [ ]:
subset_df.

In [ ]:
subset_df.head(10)

In [ ]:
subset_df['Level 2']

In [ ]:
subset_df[['Level 2']]

In [ ]:
subset_df[['Level 2']].index

In [ ]:
# Load libraries
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC


In [ ]:
models = []
models.append(('LRC', LogisticRegressionCV(solver='saga', multi_class='multinomial')))
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_subset, y_subset, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))


In [ ]:

# conda install -c conda-forge pycaret
from pycaret.classification import setup
from pycaret.classification import compare_models

# setup the dataset
#grid = setup(data=X_subset, target= y_subset, html=False, silent=True, verbose=False)
grid = setup(data=df3, target= df3.columns[-1], html=False, verbose=False)
# evaluate models and compare models
best = compare_models()
# report the best model
print(best)

In [ ]:
# evaluate models and compare models
best = compare_models()
# report the best model
print(best)

In [ ]:
# tune model hyperparameters
best_yune = tune_model(SGDClassifier(), n_iter=200, choose_better=True)
# report the best model
print(best_tune)


In [ ]:
#getnnz(X_subset)
#len(X_subset) # NW; sparse matrix length is ambiguous; use getnnz() or shape[0]
X_subset.shape[0] # 3190

In [ ]:
subset_df['Level 2'].value_counts(dropna = True)

In [ ]:
type(y_subset)
y_subset.shape[0]

In [ ]:
X_subset.toarray().shape

In [ ]:
X_subset.toarray()

In [ ]:
dfy = y_subset.reset_index()

In [ ]:
y_subset[0:10]

In [ ]:
# Reading PIMMART data
pim_gtin_mapped = pd.read_csv(DBFR + "PIM_Data_New_50_82Mn.csv", dtype=object)
for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD', 'ITM_ID', 'GTIN']:
    pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)

# Reading Syndigo 259K data
synd_ALL = pd.read_csv(DBFR + 'Syndigo_Final_ALL.csv', dtype='unicode') # 259k Syndigo Data
for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD', 'GTIN', 'ITM_ID', 'PMY_DPT_CD']:
    synd_ALL[i] = synd_ALL[i].astype(np.float64)

# Stripping spaces from all columns
df_obj = synd_ALL.select_dtypes(['object'])
synd_ALL[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [ ]:
syndigo_mapped = synd_ALL
pimmart = pim_gtin_mapped

In [ ]:
syndigo_mapped.drop_duplicates('GTIN', inplace = True)

In [ ]:
syndigo_mapped['ITEM_SUBCOM_text'] = \
(syndigo_mapped.VND_ECOM_DSC + ' ' + syndigo_mapped.SUBCOM_DSC).fillna('').str.lower()

In [ ]:
syndigo_mapped['Level 1'].value_counts()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(ngram_range= (1,2), max_features = 800000)

In [14]:
y[0:10]

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    0
8    6
9    7
dtype: int64

In [15]:
type(y)

pandas.core.series.Series

In [26]:
   data = {
        'ITEM_SUBCOM_text': X,
        'L1L2_val' : y
    
    }
    
   df_merg = pd.DataFrame(data)

ValueError: Per-column arrays must each be 1-dimensional

In [42]:
type(y)

pandas.core.series.Series

In [25]:
X[0:10]

array([[-10.710463  ,  15.938267  , -31.25719   , ...,   3.3275495 ,
         -8.461372  ,  19.65024   ],
       [-11.04187   ,   5.650813  ,  -9.047808  , ...,   7.60792   ,
         -6.18375   ,  -6.979801  ],
       [  7.025075  ,   1.1739403 , -12.235872  , ...,  -3.7004743 ,
          2.9117007 ,  20.121944  ],
       ...,
       [-12.850473  ,  14.910092  , -11.646476  , ...,  -8.275857  ,
         -1.029038  ,  15.030806  ],
       [ -4.7419806 ,  -7.3552904 , -11.437293  , ...,  -6.187852  ,
         -9.606904  ,   6.368182  ],
       [  0.92745423,   3.09524   , -12.499194  , ...,  -2.2895248 ,
         -0.07298493,   7.609419  ]], dtype=float32)

In [29]:
xs = pd.Series(list(X))

In [31]:
type(xs)

pandas.core.series.Series

In [33]:
xs[0:10]

0    [-10.710463, 15.938267, -31.25719, 4.2918553, ...
1    [-11.04187, 5.650813, -9.047808, 4.3786564, 8....
2    [7.025075, 1.1739403, -12.235872, 1.8468916, -...
3    [3.9563594, -1.2443018, -2.8453398, 4.597701, ...
4    [2.3601267, 0.85085166, 0.1611628, 11.792536, ...
5    [5.161315, 2.3726969, -17.204323, 11.801039, -...
6    [4.207485, 4.444371, -21.16404, 16.850855, -42...
7    [-12.850473, 14.910092, -11.646476, 13.040517,...
8    [-4.7419806, -7.3552904, -11.437293, 17.043844...
9    [0.92745423, 3.09524, -12.499194, 21.527416, 1...
dtype: object

In [39]:
x_w2vec  = X
y_w2vec  = y

In [40]:
df_merg_w2vec = df_merg.copy()

In [45]:
type(X)

numpy.ndarray